In [8]:
from dotenv import load_dotenv 
load_dotenv()

True

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
#openai.api_key = os.getenv("OPENAI_API_KEY")
llm_model = "gpt-3.5-turbo"

Model

In [19]:
from langchain_openai.chat_models import ChatOpenAI 
# To control the randomness and creativity of the generated text by an LLM, use temperature = 0.0
model = ChatOpenAI(temperature=0.0, model = "gpt-3.5-turbo") 
response = model.invoke("The weather is")
print(response.content)

 currently sunny with a few clouds in the sky. The temperature is around 75 degrees Fahrenheit with a light breeze. It's a beautiful day to be outside enjoying the sunshine.


Conversation

In [23]:
from langchain_core.messages import HumanMessage 

prompt= [HumanMessage("Where is located Toronto?")]
response = model.invoke(prompt)
print(response.content)

Toronto is located in the province of Ontario, Canada.


In [26]:
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI()
human_msg = HumanMessage("What is the capital of canada?")
system_msg= SystemMessage(
    "You are a helpful assistant that responds to questions with smile"
)

response = model.invoke([system_msg, human_msg])

print(response.content)

The capital of Canada is Ottawa! 😊


Prompt template

In [41]:
from langchain_core.prompts import ChatPromptTemplate 

system_template = "Translate the following form English into {language}. You are a helpful assistant that responds with smile." 
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),("user", "{text}")]
)
prompt = prompt_template.invoke({"language": "Persian", "text": "Welcome!"})

prompt.to_messages()
response = model.invoke(prompt)
print(response.content)

خوش آمدید! 😊


In [36]:
template_string = """Translate the text that is delimited by triple backticks into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)
#print(prompt_template.messages[0].prompt.input_variables)

customer_style = """American English in a respectful tone """

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
customer_messages = prompt_template.format_messages(style=customer_style,text=customer_email)

#print(type(customer_messages[0]))
# Call the LLM to translate to the style of the customer message
customer_response = model(customer_messages)
print(customer_response.content)

Ah, I am really annoyed that my blender lid came off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


Output Parsers

In [51]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

# build format_instructions:Parse the LLM output string into a Python dictionary
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
#print(format_instructions)

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt_template = ChatPromptTemplate.from_template(template=review_template)
print("this is prompt template::\n", prompt_template)

messages = prompt_template.format_messages(text=customer_review, 
                                format_instructions=format_instructions)
#print(messages[0].content)

response = model(messages)
print("This is response of Model::\n", response.content)

this is prompt template::
 input_variables=['format_instructions', 'text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: {text}\n\n{format_instructions}\n'), additional_kwargs={})]
This is response of Model::
 ```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```
